In [ ]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
import pymongo
from pymongo import MongoClient
import os

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Extrair MongoDB para Azure") \
    .config("spark.mongodb.read.connection.uri", os.environ["MONGODB_CONNECTION_STRING"]) \
    .getOrCreate()

# Conexão ao MongoDB para listar coleções
client = MongoClient(os.environ["MONGODB_CONNECTION_STRING"])
db = client[os.environ["MONGODB_DATABASE"]]
collections = db.list_collection_names()

# Loop para processar todas as coleções
for collection in collections:
    # Leitura da coleção no MongoDB como DataFrame
    df = spark.read \
        .format("mongodb") \
        .option("uri", f"{os.environ["MONGODB_CONNECTION_STRING"]}.{collection}") \
        .load()
    
    # Exibir esquema para depuração (opcional)
    df.printSchema()
    
    # Caminho no Data Lake para salvar
    output_path = f"{os.environ["AZURE_STORAGE_ACCOUNT_URL"]}/{collection}.parquet"
    
    # Gravação no Data Lake em formato Parquet
    df.write \
        .mode("overwrite") \
        .parquet(output_path)
    
    print(f"Coleção {collection} salva em {output_path}")

# Encerrar sessão Spark
spark.stop()


ModuleNotFoundError: No module named 'pyspark'